In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import * 

# Without Bias

In [2]:
X_train = [[-1,-1], [1,1], [-1,1], [1,-1]]
y_train = [0, 0, 1, 1]

X_test =  [[-1,-1] for x in range(10)]+[[1,1] for x in range(10)]+[[-1,1] for x in range(10)]+[[1,-1] for x in range(10)]
y_test =  [0 for x in range(20)]+[1 for x in range(20)]

In [3]:
with open('results/version7/train_xor.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version7/test_xor.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)

### Without Bias

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput, epoch_errosPhaseEncoding  = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=40, 
                                                                                                 trainingBias=False,
                                                                                                 error_by_epoch = True,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

In [ ]:
with open('results/version6/error_by_epoch_unbiased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosPhaseEncoding': epoch_errosPhaseEncoding, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [4]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS = experiment_TRAIN(X_train, 
                                                                                                y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=20, 
                                                                                                 trainingBias=False,
                                                                                                 error_by_epoch = False,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

IndexError: list index out of range

In [ ]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS]

In [ ]:
with open('results/version7/weights_unbiased_v7.json', 'w+') as fp:
    json.dump(weights, fp)

#### Test

In [ ]:
with open('results/version7/weights_unbiased_v7.json') as json_file:
    weights = json.load(json_file)

In [ ]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              repeat=10,
                              bias=False, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

In [ ]:
with open('results/version7/experiments_unbiased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)

## BIAS

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput, epoch_errosPhaseEncoding  = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=40, 
                                                                                                 trainingBias=True,
                                                                                                 error_by_epoch = True,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})
                                    

In [ ]:
with open('results/version6/error_by_epoch_biased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosPhaseEncoding': epoch_errosPhaseEncoding, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [4]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=20, 
                                                                                                 trainingBias=True,
                                                                                                 error_by_epoch = False,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

[[1.5707963267948968], [-1.5707963267948963, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[1.3921427353236404], [-1.2642355997888302, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 1.5707963267948963]]
[[1.5707963267948968], [-1.5707963267948963, 7.853981633974483]]
[[1.3923990229049141], [1.2646171299970965, 1.5707963267948968]]
[[1.5707963267948968], [-1.5707963267948963, 1.5707963267948963]]

erro HSGS 0.2548828125
erro encoding weight 0.3056640625
erro encoding input 0.251953125
erro phase encoding 1.0
[[1.0464238159644452], [-1.791219427461227, 1.5707963267948968]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.6000140508282293], [-1.2601537967688563, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 1.5707963267948963]]
[[1.056448231728681], [1.7794037991601575, 1.5707963267948968]]
[[1.5707963267948968], [-1.5707963267948

In [5]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS]

In [6]:
with open('results/version7/weights_biased_v7.json', 'w+') as fp:
    json.dump(weights, fp)

In [7]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              repeat=10,
                              bias=True, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

[[2.999070871454259], [7.985951229493493, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.9820318310772773], [4.588849645970098, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.999070871454259], [7.985951229493493, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.9820318310772773], [4.588849645970098, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.999070871454259], [7.985951229493493, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.9820318310772773], [4.588849645970098, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.999070871454259], [7.985951229493493, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.9820318310772773], [4.588849645970098, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981

In [8]:
with open('results/version7/experiments_biased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)